_Code Cell 1: jupyter helpers_

In [1]:
# jupyter helpers

# To get multiple outputs from one cell:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython import get_ipython
from IPython.display import HTML, Markdown, Image
# for presentations:
#display(HTML("<style>.container { width:100% !important; }</style>"))


def add_div(div_class='info', div_start='Tip:', 
            div_text='Some tip here', output_string=True):
    """
    Behaviour with default `output_string=True`:
    The cell is overwritten with the output, but the cell mode is still 'code',
    not 'markdown'.
    Workaround: After running the function, click on the new cell, press ESC, 
                type 'm', then run the new cell.
    If `output_string=False`, the output is displayed in an new cell with the 
    code cell visible.
    ```
    [x]
    add_div('alert-warning', 'Tip: ', 'some tip here', output_string=True)
    [x]
    <div class="alert alert-warning"><b>Tip: </b>some tip here</div>
    ```
    """
    accepted = ['info', 'warning', 'danger']
    div_class = div_class.lower()
    if div_class not in accepted:
        msg = f'<div class="alert"><b>Wrong class:&nbsp;</b> `div_start` not in: {accepted}.</div>'
        return Markdown(msg)
    
    div = f"""<div class="alert alert-{div_class}"><b>{div_start}&nbsp;&nbsp;</b>{div_text}</div>"""
    if output_string:
        return get_ipython().set_next_input(div, 'markdown')
    else:
        return Markdown(div)

    
def new_section(title='New section'):
    style = "text-align:center;background:#c2d3ef;padding:16px;color:#ffffff;font-size:2em;width:98%"
    div = f'<div style="{style}">{title}</div>'
    #return HTML('<div style="{}">{}</div>'.format(style, title))
    return get_ipython().set_next_input(div, 'markdown')


_Code Cell 2: project helpers_

In [2]:
# project helpers

import sys
from pathlib import Path
from pprint import pprint as pp

def sys_info():
    frmt = "\nListing from sys_info():\n"
    frmt += "Python ver: {}\nPython env: {}\n"
    frmt += "OS:         {}\nCurrent dir: {}\n"
    print(frmt.format(sys.version, 
                      Path(sys.prefix).name,
                      sys.platform,
                      Path.cwd()))


def add_to_sys_path(this_path, up=False):
    """
    Prepend this_path to sys.path.
    If up=True, path refers to parent folder (1 level up).
    """
    newp = Path(this_path).as_posix() # no str method (?)
    if up:
        newp = Path(this_path).parent.as_posix()

    msg = 'Path already in sys.path'
    if newp not in sys.path:
        sys.path.insert(1, newp)
        msg = 'Path added to sys.path'
    print(msg)

# if notebook inside another folder, eg ./notebooks:
nb_folder = 'notebooks'
add_to_sys_path(Path.cwd(), Path.cwd().name.startswith(nb_folder))


# Filtered dir() for method discovery:
def filter_dir(mdl, filter_str=None, start_with_str='_', exclude=True):
    """Filter dir(mdl) for method discovery.
       Input:
       :param mdl (object): module, optionally with submodule path(s), e.g. mdl.submdl1.submdl2.
       :param filter_str (str, None): filter all method names containing that string.
       :param start_with_str (str, '_'), exclude (bool, True): start_with_str and exclude work 
              together to perform search on non-dunder methods (default).
    """
    search_dir = [d for d in dir(mdl) if not d.startswith(start_with_str) == exclude]
    if filter_str is None:
        return search_dir
    else:
        filter_str = filter_str.lower()
        return [d for d in search_dir if d.lower().find(filter_str) != -1]

# alias:
fdir = filter_dir

def get_project_dirs(which: list,
                     nb_folder='notebooks',
                     use_parent=True):
    '''Create folder(s) named in `which` at the parent level.'''
    dir_lst = []
    if Path.cwd().name.startswith(nb_folder) or use_parent:
        dir_fn = Path.cwd().parent.joinpath
    else:
        dir_fn = Path.cwd().joinpath
        
    for d in which:
        DIR = dir_fn(d)
        if not DIR.exists():
            Path.mkdir(DIR)
        dir_lst.append(DIR)
    return dir_lst

#DIR_DATA, 
DIR_IMG, DIR_TMP = get_project_dirs(['images', '_temp'])


# autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    get_ipython().run_line_magic('load_ext', 'autoreload')

%autoreload 2

#..................
sys_info()

no_wmark = False
try:
    %load_ext watermark
    #%watermark
except ModuleNotFoundError:
    no_wmark = True

if not no_wmark:
    print("\nListing from watermark ext:")
    %watermark
    #-iv

Path added to sys.path

Listing from sys_info():
Python ver: 3.10.8 | packaged by conda-forge | (main, Nov 22 2022, 08:16:33) [MSC v.1929 64 bit (AMD64)]
Python env: miniconda3
OS:         win32
Current dir: c:\Users\catch\Documents\GitHub\new_conda_env\notebooks


Listing from watermark ext:
Last updated: 2023-02-28T12:12:17.808497-05:00

Python implementation: CPython
Python version       : 3.10.8
IPython version      : 8.9.0

Compiler    : MSC v.1929 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 142 Stepping 10, GenuineIntel
CPU cores   : 8
Architecture: 64bit



---
---
# Testing modules
---

# TODO:
## test logging level
* Wanted: the cli sets the level
=> remove root logger in init
=> write a test fn to findout which is the level when cli:main is run

In [5]:
from new_conda_env import cli

In [26]:
p = cli.generate_parser()
p

argv = ["-old_ver", "3.10" , "-new_ver", "3.9", "-new_env_name", "ds39", "-env_to_clone", "ds310"]
args = p.parse_args(argv)
args

ArgumentParser(prog='new_conda_env', usage=None, description='Main module (cli):\nconda_new_env is a tool for \'lean cloning\' an environment from \nan existing one, e.g. when a new kernel version is desired. \nThe output is a yml file without dependencies versions except for that \nof the (python) kernel and includes any pip dependencies."\n', formatter_class=<class 'argparse.ArgumentDefaultsHelpFormatter'>, conflict_handler='error', add_help=True)

In [70]:
class CondaFlag:
    NOBLD = "--no-builds"
    HIST = "--from-history"


flag = CondaFlag.HIST
flag

'--from-history'

# Testing cli:
1. fn to mock: main : by-pass conda_vir.create_new_env_yaml()
2. class to mock: envir.CondaEnvir

In [10]:
from new_conda_env import envir, processing as proc
from new_conda_env.cli import generate_parser, main
from unittest.mock import patch
import pytest

In [26]:
kwargs_req = ["old_ver", "new_ver", "env_to_clone"]
kwargs_req_vals = ["3.10" ,  "3.9", "ds310"]
kwargs_req_expect = ["3.10" ,  "39", "ds310"]

kwagrs_opts = ["dotless_ver", "new_env_name", "kernel", "display_new_yml", "log_level"]
kwagrs_opts_def_vals = [1, "default", "python", 1, "DEBUG"]
kwagrs_opts_fail_vals = [1, "default", "R", 1, "DEBUG"]

dict_reqs = dict(zip(kwargs_req, kwargs_req_vals))
dict_ops_ok = dict(zip(kwagrs_opts, kwagrs_opts_def_vals))
dict_ops_fail = dict(zip(kwagrs_opts, kwagrs_opts_fail_vals))

In [49]:
from conda.base.context import context

In [59]:
context.envs_dirs
#"env_dir": Path(context.envs_dirs[0]),
old_prefix = CE.basic_info["env_dir"].joinpath("ds310")
old_prefix

('c:\\Users\\catch\\miniconda3\\envs',
 'C:\\Users\\catch\\.conda\\envs',
 'C:\\Users\\catch\\AppData\\Local\\conda\\conda\\envs')

WindowsPath('c:/Users/catch/miniconda3/envs/ds310')

In [53]:
CE = envir.CondaEnvir(**dict_reqs)

In [68]:
CE.get_export_cmd("ds310", CondaFlag.HIST)

'conda env export -n ds310 --from-history'

In [55]:
def test_get_user_rc(CE):
    rc = CE.get_user_rc()
    if rc is not None:
        expected = CE.basic_info['conda_prefix'].parent.joinpath(".condarc")
        assert rc == expected

test_get_user_rc(CE)


def test_get_export_cmd():
    """
    env_to_clone = 'ds310'
    flag = CondaFlag.HIST
    
    user_dir = Path().home()
    yml_file = f"env_{env_to_clone}_{fla.name.lower()}.yml"
    yml_path = proc.path2str(user_dir.joinpath(yml_file))
        
    out = get_export_cmd(env_to_clone, flag, yml_path)
    """

In [66]:
methods_to_test = fdir(CE)

excluded = ["get_rc_python_deps", "get_export_stream","get_new_env_yaml","has_user_rc", "get_user_rc","user_rc"]

for ex in excluded:
    _ = methods_to_test.pop(methods_to_test.index(ex))

methods_to_test

['basic_info',
 'conda_root',
 'display_new_yml',
 'dotless_ver',
 'env_to_clone',
 'get_conda_info',
 'get_export_cmd',
 'get_lean_yml_pathname',
 'get_new_env_name',
 'kernel',
 'log',
 'new_env_name',
 'new_prefix',
 'new_ver',
 'new_yml',
 'old_ver',
 'user_dir']

In [48]:
for m in methods_to_test:
    print(m,"::\n", getattr(CE, m))


basic_info ::
 {'conda_prefix': WindowsPath('c:/Users/catch/miniconda3'), 'active_prefix': WindowsPath('C:/Users/catch/miniconda3'), 'user_condarc': WindowsPath('C:/Users/catch/.condarc'), 'env_dir': WindowsPath('c:/Users/catch/miniconda3/envs'), 'default_python': '3.10'}
conda_root ::
 C:\Users\catch\miniconda3
create_new_env_yaml ::
 <bound method CondaEnvir.create_new_env_yaml of CondaEnvir(old_ver: str = '', new_ver: str = '', dotless_ver: bool = True, env_to_clone: str = '', new_env_name: str = 'default', kernel: str = 'python', display_new_yml: bool = True, log_level: str = 'ERROR')>
display_new_yml ::
 True
dotless_ver ::
 True
env_to_clone ::
 ds310
get_conda_info ::
 <bound method CondaEnvir.get_conda_info of CondaEnvir(old_ver: str = '', new_ver: str = '', dotless_ver: bool = True, env_to_clone: str = '', new_env_name: str = 'default', kernel: str = 'python', display_new_yml: bool = True, log_level: str = 'ERROR')>
get_export_cmd ::
 <function CondaEnvir.get_export_cmd at 0x0

In [14]:
import logging

In [15]:
levels = logging._nameToLevel
choices = list(levels.keys())
choices

['CRITICAL', 'FATAL', 'ERROR', 'WARN', 'WARNING', 'INFO', 'DEBUG', 'NOTSET']

In [16]:
logr_envir = logging.getLogger("this")
logr_envir.setLevel(logging.ERROR)
logr_envir.level

40

In [17]:
logr_envir.error("oops")

oops


In [18]:
del logr_envir
logr_envir = logging.getLogger("this")
logr_envir.setLevel("ERROR")
logr_envir.level

40

# No dry-run for conda env create!
### see [Issue7495](https://github.com/conda/conda/issues/7495)

In [71]:
from new_conda_env import envir, processing as proc
from new_conda_env.cli import generate_parser

In [75]:
conda_env_parser = generate_parser()
conda_env_parser.prog
print(conda_env_parser.description)

args_lst = "-old_ver 3.10.8 -new_ver 3.9 -dotless_ver 1 -env_to_clone ds310 -new_env_name ds39".split()
args_lst 
args = conda_env_parser.parse_args(args_lst)
args
args.env_to_clone

'new_conda_env'

Main module (cli):
conda_new_env is a tool for 'lean cloning' an environment from 
an existing one, e.g. when a new kernel version is desired. 
The output is a yml file without dependencies versions except for that 
of the (python) kernel and includes any pip dependencies."



['-old_ver',
 '3.10.8',
 '-new_ver',
 '3.9',
 '-dotless_ver',
 '1',
 '-env_to_clone',
 'ds310',
 '-new_env_name',
 'ds39']

Namespace(old_ver='3.10.8', new_ver='3.9', dotless_ver=True, env_to_clone='ds310', new_env_name='ds39', kernel='python', display_new_yml=1, log_level='ERROR')

'ds310'

In [76]:
conda_vir = envir.CondaEnvir(old_ver=args.old_ver,
                             new_ver=args.new_ver, 
                             dotless_ver=args.dotless_ver,
                             env_to_clone=args.env_to_clone,
                             new_env_name=args.new_env_name,
                             kernel=args.kernel,
                             display_new_yml=args.display_new_yml,
                             log_level=args.log_level)

fdir(conda_vir)

msg = "Basic info:\n"
msg = msg + f"- Root: {conda_vir.conda_root}\n"
msg = msg + f"- Info dict:\n\t{conda_vir.basic_info}"
print(msg)

['basic_info',
 'conda_root',
 'display_new_yml',
 'dotless_ver',
 'env_to_clone',
 'get_conda_info',
 'get_export_cmd',
 'get_export_stream',
 'get_lean_yml_pathname',
 'get_new_env_name',
 'get_new_env_yaml',
 'get_rc_python_deps',
 'get_user_rc',
 'has_user_rc',
 'kernel',
 'log',
 'new_env_name',
 'new_prefix',
 'new_ver',
 'new_yml',
 'old_ver',
 'user_dir',
 'user_rc']

Basic info:
- Root: C:\Users\catch\miniconda3
- Info dict:
	{'conda_prefix': WindowsPath('c:/Users/catch/miniconda3'), 'active_prefix': WindowsPath('C:/Users/catch/miniconda3'), 'user_condarc': WindowsPath('C:/Users/catch/.condarc'), 'env_dir': WindowsPath('c:/Users/catch/miniconda3/envs'), 'default_python': '3.10'}


# Call example at the cmd or terminal window:
`python -m new_conda_env.main -old_ver 3.10 -new_ver 3.9 -dotless_ver 1 -env_to_clone ds310 -new_env_name ds39`

# After publishing on conda-forge:
`new_conda_env -old_ver 3.10 -new_ver 3.9 -dotless_ver 1 -env_to_clone ds310 -new_env_name ds39`
